In [ ]:
pip install syft==0.2.9

     |████████████████████████████████| 433 kB 6.5 MB/s 
     |████████████████████████████████| 1.8 MB 50.0 MB/s 
     |████████████████████████████████| 57 kB 7.2 MB/s 
     |████████████████████████████████| 126 kB 59.9 MB/s 
     |████████████████████████████████| 9.0 MB 15.7 MB/s 
     |████████████████████████████████| 73 kB 2.5 MB/s 
     |████████████████████████████████| 66 kB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 61.8 MB/s 
     |████████████████████████████████| 2.0 MB 38.5 MB/s 
     |████████████████████████████████| 200 kB 17.8 MB/s 
     |████████████████████████████████| 484 kB 56.3 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 79 kB 9.8 MB/s 
     |████████████████████████████████| 753.4 MB 6.7 kB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 449 kB 57.3 MB/s 
     |████████████████████████████████| 75 kB 4.2 MB/s 
     |████████████████████

In [ ]:
import torch
import torchvision
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import syft as sy
hook = sy.TorchHook(torch)

**Create Virtual devices**

In [ ]:
jake = sy.VirtualWorker(hook, id="jake") #creating a virtual worker jake (analogous to client device 1)
john = sy.VirtualWorker(hook, id="john") #creating a virtual worker john (analogous to client device 2)

**Send data to client devices**

In [ ]:

transform = transforms.Compose([ #combines all the transforms that are given to it (ex: transformation 1 and transformation 2)
    transforms.ToTensor(), #converts an image to a tensor  #transformation 1
    transforms.Normalize((0.5, ), (0.5, )), #Normalize an image with mean and standard deviation #transformation 2
])

train_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=True, download=True, transform=transform) #download the dataset for training(MNIST) 
test_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=False, download=True, transform=transform) #download the dataset for testing

federated_train_loader = sy.FederatedDataLoader(
    train_set.federate((jake, john)), batch_size=64, shuffle=True) #takes a federated dataset and returns an iterator over it

#train.set.federate((jake, john)) --> this will make the dataset(train) federated and send them to the workers jake and john

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=64, shuffle=True) #takes a dataset(test) and provides and iterator over it

0it [00:00, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


0it [00:00, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


0it [00:00, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


0it [00:00, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw
Processing...
Done!


**Building the Model**

In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 500) #784 neurons in input layer, 500 in hidden layer, #fc1 will contain the weights and biases of the model
        self.fc2 = nn.Linear(500, 10)  #10 neurons in the output layer #fc2 will contain the weights and biases of the model
        # we can use weight, bias = list(fc1.parameters()) to see those weights and parameters returned by the model #we can use these weights and biases in conjunction with blockchain

    def forward(self, x):
        x = x.view(-1, 784) #flattening the images in rank 2 tensor
        x = self.fc1(x) #making predictions
        x = F.relu(x)   #applying relu activation function
        x = self.fc2(x) #making predictions
        return F.log_softmax(x, dim=1)


model = Model()
optimizer = optim.SGD(model.parameters(), lr=0.01)

**Training the Model**

Since the data is present on the client device, we obtain its location through the *location* attribute.

In [ ]:
for epoch in range(0, 5):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): #federated_train_loader is an iterator over the dataset available at both the clients
        model.send(data.location)# send the model to the client device where the data is present, with each iteration model is being sent to the client
        # training the model
        optimizer.zero_grad() #clears the gradients
        output = model(data) 
        loss = F.nll_loss(output, target) #getting the loss (using negative log likelihood loss function)
        loss.backward() #computing update for each parameter
        optimizer.step() #making the update for each parameter
        
        model.get() # get back the improved model
        if batch_idx % 100 == 0:
            # get back the loss
            loss = loss.get()
            print('Epoch: {:2d} [{:5d}/{:5d} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                batch_idx * 64,
                len(federated_train_loader) * 64,
                100. * batch_idx / len(federated_train_loader),
                loss.item()))

Epoch:  1 [    0/60032 (  0%)]	Loss: 2.341757
Epoch:  1 [ 6400/60032 ( 11%)]	Loss: 1.216436
Epoch:  1 [12800/60032 ( 21%)]	Loss: 0.835575
Epoch:  1 [19200/60032 ( 32%)]	Loss: 0.713210
Epoch:  1 [25600/60032 ( 43%)]	Loss: 0.523373
Epoch:  1 [32000/60032 ( 53%)]	Loss: 0.478899
Epoch:  1 [38400/60032 ( 64%)]	Loss: 0.256966
Epoch:  1 [44800/60032 ( 75%)]	Loss: 0.340812
Epoch:  1 [51200/60032 ( 85%)]	Loss: 0.457695
Epoch:  1 [57600/60032 ( 96%)]	Loss: 0.330051
Epoch:  2 [    0/60032 (  0%)]	Loss: 0.289611
Epoch:  2 [ 6400/60032 ( 11%)]	Loss: 0.302198
Epoch:  2 [12800/60032 ( 21%)]	Loss: 0.569484
Epoch:  2 [19200/60032 ( 32%)]	Loss: 0.385424
Epoch:  2 [25600/60032 ( 43%)]	Loss: 0.401111
Epoch:  2 [32000/60032 ( 53%)]	Loss: 0.340114
Epoch:  2 [38400/60032 ( 64%)]	Loss: 0.309766
Epoch:  2 [44800/60032 ( 75%)]	Loss: 0.309509
Epoch:  2 [51200/60032 ( 85%)]	Loss: 0.242860
Epoch:  2 [57600/60032 ( 96%)]	Loss: 0.305040
Epoch:  3 [    0/60032 (  0%)]	Loss: 0.307606
Epoch:  3 [ 6400/60032 ( 11%)]	Los

**Testing the model**

In [ ]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += F.nll_loss(
            output, target, reduction='sum').item() #getting the total loss(summing up for each iteration) for test dataset using negative log likelihood loss(NLL) function
        # get the index of the max log-probability
        pred = output.argmax(1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item() #for correctly classified image (equates the true target value with the pred)

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss,
    correct,
    len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.2512, Accuracy: 9285/10000 (93%)

